In [2]:
# Connect the runtime environment to GooGle Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load and split the image dataset.
import tensorflow as tf
import os

# Save the link to the google drive folder with the images.
base_directory = "/content/drive/MyDrive/PersonalFiles/MaizeLeafNutrientDeficiencyDatasets"

data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1.0 / 255.0,
    validation_split = 0.15
) 

batchsize = 64

training_data = data_generator.flow_from_directory(
    base_directory,
    target_size = (224, 224),
    batch_size = batchsize,
    subset = 'training'
)

testing_data = data_generator.flow_from_directory(
    base_directory,
    target_size = (224, 224),
    batch_size = batchsize,
    subset = 'validation'
)

Found 5436 images belonging to 2 classes.
Found 958 images belonging to 2 classes.


In [ ]:
# Identify the indices of the different classes.
training_data.class_indices

{'Healthy': 0, 'NitrogenDeficient': 1}

In [ ]:
# Design the architecture and structure of the image classification model.
maize_nitrogen_deficiency_detection_model = tf.keras.models.Sequential(
    [
     tf.keras.layers.Conv2D(
         64,
         (3, 3),
         activation = 'relu',
         input_shape = (224, 224, 3)
     ),
     tf.keras.layers.MaxPooling2D(2, 2),
     tf.keras.layers.Conv2D(
         64,
         (3, 3),
         activation = 'relu'
     ),
     tf.keras.layers.MaxPooling2D(2, 2),
     tf.keras.layers.Dropout(0.5),
     tf.keras.layers.Conv2D(
         64,
         (3, 3),
         activation = 'relu'
     ),
     tf.keras.layers.MaxPooling2D(2, 2),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(
         1024,
         activation = 'relu'
     ),
     tf.keras.layers.Dropout(0.5),
     tf.keras.layers.Dense(
         2,
         activation = 'softmax'
     )
    ]
)

In [ ]:
# Compile your newly deigned image classification mode.
maize_nitrogen_deficiency_detection_model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'sgd',
    metrics = ['accuracy']
)

In [ ]:
# Establish the number of training and testing / validation steps in each epoch.
training_steps = training_data.samples // batchsize
testing_steps = testing_data.samples // batchsize

In [ ]:
# Build the model.
maize_nitrogen_deficiency_detection_model.fit(
    training_data,
    validation_data = testing_data,
    epochs = 25,
    steps_per_epoch = training_steps,
    validation_steps = testing_steps,
)

Epoch 1/25
79/79 [==============================] - 1300s 16s/step - loss: 0.4676 - accuracy: 0.7654 - val_loss: 0.7114 - val_accuracy: 0.3594
Epoch 2/25
79/79 [==============================] - 113s 1s/step - loss: 0.3750 - accuracy: 0.8276 - val_loss: 0.8527 - val_accuracy: 0.2936
Epoch 3/25
79/79 [==============================] - 109s 1s/step - loss: 0.3282 - accuracy: 0.8478 - val_loss: 0.2167 - val_accuracy: 0.9770
Epoch 4/25
79/79 [==============================] - 111s 1s/step - loss: 0.2633 - accuracy: 0.8876 - val_loss: 0.5610 - val_accuracy: 0.6637
Epoch 5/25
79/79 [==============================] - 113s 1s/step - loss: 0.2829 - accuracy: 0.8824 - val_loss: 0.2820 - val_accuracy: 0.9342
Epoch 6/25
79/79 [==============================] - 112s 1s/step - loss: 0.2142 - accuracy: 0.9145 - val_loss: 0.1613 - val_accuracy: 0.9646
Epoch 7/25
79/79 [==============================] - 110s 1s/step - loss: 0.1793 - accuracy: 0.9268 - val_loss: 0.1144 - val_accuracy: 0.9671
Epoch 8/25


In [ ]:
# Save the tensorflow model.
maize_nitrogen_deficiency_detection_model.save("maizenitrogendeficiencydetectionmodel.h5")

In [ ]:
# Get the summary of the model.
maize_nitrogen_deficiency_detection_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 54, 54, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        3

In [ ]:
# Evaluate the testing / validation loss and accuracy of the model.
loss, accuracy = maize_nitrogen_deficiency_detection_model.evaluate(testing_data)

20/20 [==============================] - 298s 15s/step - loss: 0.7078 - accuracy: 0.2340


In [ ]:
# Evaluate the training loss and accuracy of the model.
loss, accuracy = maize_nitrogen_deficiency_detection_model.evaluate(training_data)

80/80 [==============================] - 1208s 15s/step - loss: 0.7081 - accuracy: 0.2518


In [ ]:
# Get the model indices.
training_data.class_indices

{'Healthy': 0, 'NitrogenDeficient': 1}

In [ ]:
# Test the model by importing images for evaluation.
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files

uploaded_files = files.upload()

for path in uploaded_files.keys():
  input_image = image.load_img(path, target_size = (224, 224))
  image_values = image.img_to_array(input_image)
  dimension_corrected = np.expand_dims(image_values, axis = 0)
  prediction = maize_nitrogen_deficiency_detection_model.predict(dimension_corrected)
  argmax_value = np.argmax(prediction)
  print(argmax_value)

Saving ndmaizeimages.jpg to ndmaizeimages.jpg
0


In [ ]:
# Convert the keras tensorflow model to tensorflow lite.
converter = tf.lite.TFLiteConverter.from_keras_model(maize_nitrogen_deficiency_detection_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp81nihiah/assets


In [ ]:
# Save the newly generated tensorflow lite model.
with open('maize_nitrogen_deficiency_detection_model.tflite', 'wb') as f:
  f.write(tflite_model)